In [2]:
import re, os, sys
from functools import reduce
from itertools import combinations as comb, permutations as perm, combinations_with_replacement as combr
from operator import itemgetter
from pprint import pprint
from math import *
from collections import defaultdict
import networkx as nx
import numpy as np

def dprint(d):
    pprint(dict(d))
def lmap(*args, **kwargs):
    return list(map(*args, **kwargs))
def lmapi(*args, **kwargs):
    return list(map(int, *args, **kwargs))
def enum(*args, **kwargs):
    return enumerate(*args, **kwargs)

In [85]:
input_fname = "../inputs/day22-demo.in"

In [3]:
input_fname = "../inputs/day22.in"

In [86]:
def reread():
    lines = []
    with open(input_fname) as ifile:
        def r(): return ifile.readline().strip()
        def rs(): return ifile.readline().strip().split()
        def ri(): return int(ifile.readline().strip())
        def ris(): return list(map(int, ifile.readline().strip().split()))

        rows = []
        maxwidth = 0
        # read map
        for line in ifile:
            if not line.strip(): break
            row = [" .#".index(c) for c in line.rstrip()]
            rows.append(row)
            maxwidth = max(maxwidth, len(row))

        ar = np.zeros((len(rows), maxwidth), dtype=int)
        for i, row in enum(rows):
            ar[i, :len(row)]  = row
        # directions
        path = r()
        path = path.replace("R", " R ").replace("L", " L ").split()
    
    return ar, path

ar, path = reread()
# ar[:10], path[:10]

## Part 1

In [87]:
rowlos = []
rowhis = []
for row in ar:
    idx = np.nonzero(row > 0)
    rowlos.append(np.min(idx))
    rowhis.append(np.max(idx))
# rowlos, rowhis

In [88]:
collos = []
colhis = []
for col in ar.T:
    idx = np.nonzero(col > 0)
    collos.append(np.min(idx))
    colhis.append(np.max(idx))
# collos, colhis

In [89]:
def i2c(cn):
    return int(cn.real), int(cn.imag)
def c2i(x, y):
    return x+1j*y

In [90]:
def travel(strip, relpos, dist):
    # print("travel got", strip, relpos, dist)
    if np.max(strip) == 1:  # no wall
        newpos = relpos + dist
        # print(f"no wall")#, {pos}->{newpos}")
    else:  # some wall
        rem = np.hstack((strip, strip))[relpos:relpos+len(strip)]
        # print(np.max(strip), rem, (rem == 2).nonzero())
        walldist = (rem == 2).nonzero()[0][0]
        if dist >= walldist:
            # hit wall
            newpos = relpos + walldist - 1
            # print(f"hit wall")#, {pos}->{newpos}")
        else:
            # no hit
            newpos = relpos + dist
            # print(f"free move")#, {pos}->{newpos}")
    return newpos % len(strip)

In [91]:
pos = c2i(rowlos[0], collos[rowlos[0]])
face = 1+0j
for move in path:
    if move == "R":
        face *= 1j
        # print("turning right", face)
    elif move == "L":
        face *= -1j
        # print("turning left", face)
    else:
        dist = int(move)
        x, y = i2c(pos)
        if face == 1:  # right
            row = ar[y][ar[y]>0]
            lo = rowlos[y]
            xrel = x - lo
            newx = lo + travel(row, xrel, dist)
            newpos = c2i(newx, y)
        elif face == -1:  # left
            row = ar[y][ar[y]>0][::-1]
            hi = rowhis[y]
            xrel = hi - x
            newx = hi - travel(row, xrel, dist)
            newpos = c2i(newx, y)
        elif face == 1j:  # down
            col = ar[:,x][ar[:,x]>0]
            lo = collos[x]
            yrel = y - lo
            newy = lo + travel(col, yrel, dist)
            newpos = c2i(x, newy)
        elif face == -1j:  # up
            col = ar[:,x][ar[:,x]>0][::-1]
            hi = colhis[x]
            yrel = hi - y
            newy = hi - travel(col, yrel, dist)
            newpos = c2i(x, newy)
        else:
            raise RuntimeError(f"shouldn't reach here, face: {face}")
        # print(f"updating pos, {i2c(pos)}->{i2c(newpos)}")
        pos = newpos
    # break
print("final:", pos)
password = 0
password += (pos.imag + 1)*1000
password += (pos.real + 1)*4
password += [1, -1j, -1, 1j].index(face)
print("password:", int(password))

final: (7+5j)
password: 6032


## Part 2

# Try next

> probably need to encode 3 bits of information and complex number isn't enough
> - next side
> - flip or not (`DIM - 1 - yin`)
> - next face on next side (includes `reverse` or not, only `-1` and `-1j` are reverse)

In [111]:
# small params
DIM = 4
SIDENUMS = np.array([
    [0, 0, 1, 0],
    [2, 3, 4, 0],
    [0, 0, 5, 6]
])

NBRS = {
     -1: {1: 3j, 2:-6j, 3:  2, 4:  3, 5: 3j, 6:  5},  # left
      1: {1: -6, 2:  3, 3:  4, 4:-6j, 5:  6, 6: -1},  # right
    -1j: {1: -2, 2: -1, 3: 1j, 4:  1, 5:  4, 6:-4j},  # up
     1j: {1:  4, 2: -5, 3:-5j, 4:  5, 5: -2, 6:-2j},  # down
}

In [38]:
# large params
DIM = 50
SIDENUMS = np.array([
    [0, 2, 1],
    [0, 3, 0],
    [5, 4, 0],
    [6, 0, 0]
])

NBRS = {
     -1: {1:  2, 2: -5, 3: 5j, 4:  5, 5: -2, 6: 2j},  # left
      1: {1: -4, 2:  1, 3: 1j, 4: -1, 5:  4, 6: 4j},  # right
     1j: {1:  6, 2: 6j, 3:  2, 4:  3, 5: 3j, 6:  5},  # up
    -1j: {1: 3j, 2:  3, 3:  4, 4: 6j, 5:  6, 6:  1},  # down
}

In [99]:
def find_side(pos):
    # print(i2c(pos), end="->")
    pos /= DIM
    # print(i2c(pos), end="\t")
    x, y = i2c(pos)
    # print("x", x, "y", y, "\t", sides[y,x])
    return SIDENUMS[y,x]

In [100]:
# debug cube
nonzeros = list(zip(*ar.nonzero()))
ctr = 1
for y, x in nonzeros:
    ar[y,x] = ctr
    ctr += 1
# ar

```
                            +-------------+
                            |  1  2  3  4 |           
                            |  5  6  7  8 |           
                            |  9 10 11 12 |           
                            | 13 14 15 16 |           
+-------------+-------------+-------------+
| 17 18 19 20 | 21 22 23 24 | 25 26 27 28 |           
| 29 30 31 32 | 33 34 35 36 | 37 38 39 40 |           
| 41 42 43 44 | 45 46 47 48 | 49 50 51 52 |           
| 53 54 55 56 | 57 58 59 60 | 61 62 63 64 |
+-------------+-------------+-------------+-------------+         
                            | 65 66 67 68 | 69 70 71 72 |
                            | 73 74 75 76 | 77 78 79 80 |
                            | 81 82 83 84 | 85 86 87 88 |
                            | 89 90 91 92 | 93 94 95 96 |
                            +-------------+-------------+
```

In [101]:
SIDES = dict()
for sy, srow in enum(SIDENUMS):
    for sx, sidenum in enum(srow):
        if sidenum:
            SIDES[sidenum] = ar[sy*DIM:(sy+1)*DIM, sx*DIM:(sx+1)*DIM]
            # print("extracted side", sidenum)
            # print(ar[sy*DIM:(sy+1)*DIM, sx*DIM:(sx+1)*DIM])

In [109]:
def collect_strips(ar, pos, face):
    def isneg(cn):
        return cn.real < 0 or cn.imag < 0
    x, y = i2c(pos)
    xin, yin = x%DIM, y%DIM
    side = find_side(pos)
    
    strips = []
    for i in range(4):
        if face.imag == 0:  # horizontal strip
            strip = SIDES[side][yin]
        else:  # vertical strip
            strip = SIDES[side][:,xin]
        if isneg(face): strip = strip[::-1]
        print("strip", i, strip)
        strips.append(strip)
        print()
        
        # prepare for next strip
        nxt = NBRS[face][side]
        nxtside = int(abs(nxt))
        nxtface = nxt/nxtside
        side = nxtside
        print("face", face, "\tnxtface", nxtface, "\tprod", face*nxtface)
        face *= nxtface
        if isneg(nxtface):
            yin = (DIM - 1) - yin
            xin = (DIM - 1) - xin
        print("next side", nxtside, "\tface", face, "\tyin", yin)
    return np.hstack(strips)

In [110]:
#collect_strips(ar, 8+0j, 1)
#1,2,3,4,96,95,94,93,92,91,90,89,57,45,33,21
collect_strips(ar, 15+11j, -1)

strip 0 [96 95 94 93]

face -1 	nxtface 1.0 	prod -1.0
next side 5 	face -1.0 	yin 3
strip 1 [92 91 90 89]

face -1.0 	nxtface 1j 	prod (-0-1j)
next side 3 	face (-0-1j) 	yin 3
strip 2 [60 48 36 24]

face (-0-1j) 	nxtface 1j 	prod (1-0j)
next side 1 	face (1-0j) 	yin 3
strip 3 [13 14 15 16]

face (1-0j) 	nxtface -1.0 	prod (-1+0j)
next side 6 	face (-1+0j) 	yin 0


array([96, 95, 94, 93, 92, 91, 90, 89, 60, 48, 36, 24, 13, 14, 15, 16])

In [40]:
def extract_strip(ar, pos, face):
    x, y = i2c(pos)
    xin, yin = x%DIM, y%DIM
    side = find_side(pos)
    if face == 1:
        # strip = []
        # curside = side
        # curface = face
        # curyin = yin
        # reverse = False
        # for i in range(4):
        #     curstrip = SIDES[curside][curyin]
        #     if reverse: curstrip = curstrip[::-1]
        #     print("strip", i, curstrip)
        #     strip = np.hstack((strip, curstrip))
        #     # prepare for next strip
        #     nextside = NBRS[curface][curside]
        #     print("next side", nextside)
        #     if nextside < 0:
        #         curface = -curface
        #         curside = -curside
        #         curyin = (DIM - 1) - curyin
        #         reverse = not reverse

        strip0 = SIDES[side][yin]
        print("strip 0", strip0)
        side1 = NBRS[face][side]
        print("side1", side1)
        if side1 < 0:
            face1 = -face
            side1 = -side1
            yin = (DIM - 1) - yin
            strip1 = SIDES[side1][yin][::-1]
        else:
            strip1 = SIDES[side1][yin]
        print("strip 1", strip1)
        side2 = NBRS[face1][side1]
        print("side2", side2)
        if side2 < 0:
            face2 = -face1
            side2 = -side2
            yin = (DIM - 1) - yin
            print("side2 yin(rev)", yin)
            strip2 = SIDES[side2][yin][::-1]
        else:
            print("side2 yin", yin)
            strip2 = SIDES[side2][yin]
        print("strip 2", strip2)


In [42]:
extract_strip(ar, 8+0j, 1)

strip 0 [1 1 1 2]
side1 -6
strip 1 [1 2 1 1]
side2 5
side2 yin 3
strip 2 [1 2 1 1]


In [12]:
def extract_vstrip(ar, pos, face):
    if face == -1j:  # down
        col = ar[:,x][ar[:,x]>0]
        lo = collos[x]
        yrel = y - lo
        newy = lo + travel(col, yrel, dist)
    elif face == 1j:  # up
        col = ar[:,x][ar[:,x]>0][::-1]
        hi = colhis[x]
        yrel = hi - y
        newy = hi - travel(col, yrel, dist)
    else:
        raise ValueError(f"extract_vstrip got face {face}")
    return c2i(x, newy)

In [23]:
pos = c2i(rowlos[0], collos[rowlos[0]])
face = 1+0j
for move in path:
    if move == "R":
        face *= -1j
        # print("turning right", face)
    elif move == "L":
        face *= 1j
        # print("turning left", face)
    else:
        dist = int(move)
        x, y = i2c(pos)
        if face == 1:  # right/left
            row = ar[y][ar[y]>0]
            lo = rowlos[y]
            xrel = x - lo
            newx = lo + travel(row, xrel, dist)
            newpos = c2i(newx, y)
        elif face == -1:  # left
            row = ar[y][ar[y]>0][::-1]
            hi = rowhis[y]
            xrel = hi - x
            newx = hi - travel(row, xrel, dist)
            newpos = c2i(newx, y)
        elif face == -1j:  # down
            col = ar[:,x][ar[:,x]>0]
            lo = collos[x]
            yrel = y - lo
            newy = lo + travel(col, yrel, dist)
            newpos = c2i(x, newy)
        elif face == 1j:  # up
            col = ar[:,x][ar[:,x]>0][::-1]
            hi = colhis[x]
            yrel = hi - y
            newy = hi - travel(col, yrel, dist)
            newpos = c2i(x, newy)
        else:
            raise RuntimeError(f"shouldn't reach here, face: {face}")
        print(f"updating pos, {i2c(pos)}->{i2c(newpos)}")
        find_side(newpos)
        pos = newpos
    # break
print("final:", pos)
password = 0
password += (pos.imag + 1)*1000
password += (pos.real + 1)*4
password += [1, -1j, -1, 1j].index(face)
print("password:", int(password))

updating pos, (8, 0)->(10, 0)
(10, 0)->(2, 0)	x 2 y 0 	 1
updating pos, (10, 0)->(10, 5)
(10, 5)->(2, 1)	x 2 y 1 	 4
updating pos, (10, 5)->(3, 5)
(3, 5)->(0, 1)	x 0 y 1 	 2
updating pos, (3, 5)->(3, 7)
(3, 7)->(0, 1)	x 0 y 1 	 2
updating pos, (3, 7)->(7, 7)
(7, 7)->(1, 1)	x 1 y 1 	 3
updating pos, (7, 7)->(7, 5)
(7, 5)->(1, 1)	x 1 y 1 	 3
updating pos, (7, 5)->(7, 5)
(7, 5)->(1, 1)	x 1 y 1 	 3
final: (7+5j)
password: 6032
